In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('/content/drive/MyDrive/Dados/Projetos/UCL_2425/ucl2425.csv')

In [31]:
# Agrupa os dados por partida e time, somando os toques no terço final
touches_by_match_team = df.groupby(['Match_ID', 'Squad'])['Touches_Att 3rd'].sum().reset_index()

# Calcula o total de toques de cada partida (somando os dois times)
total_touches_by_match = touches_by_match_team.groupby('Match_ID')['Touches_Att 3rd'].sum().reset_index()

# Renomeia a coluna para evitar conflito no merge
total_touches_by_match = total_touches_by_match.rename(columns={'Touches_Att 3rd': 'Total_Touches_Att3rd'})

# Une os dados para ter o total de toques e os toques do time na mesma linha
field_tilt_df = pd.merge(touches_by_match_team, total_touches_by_match, on='Match_ID')

# Calcula o Field Tilt
field_tilt_df['Field_Tilt'] = field_tilt_df['Touches_Att 3rd'] / field_tilt_df['Total_Touches_Att3rd']


In [41]:
#field_tilt_df[['Match_ID', 'Squad', 'Field_Tilt']]

field_tilt_df.head()

,Match_ID,Squad,Touches_Att 3rd,Total_Touches_Att3rd,Field_Tilt
0,1,Aston Villa,162,258,0.627907
1,1,Young Boys,96,258,0.372093
2,2,Juventus,89,309,0.288026
3,2,PSV Eindhoven,220,309,0.711974
4,3,Lille,119,233,0.510730


In [33]:
def calculate_field_tilt(df):
    """
    Calcula o Field Tilt para cada time em cada partida e o Field Tilt acumulado até a rodada atual.
    """

    # Calcula o Field Tilt por partida (como antes)
    touches_by_match_team = df.groupby(['Match_ID', 'Squad', 'MD'])['Touches_Att 3rd'].sum().reset_index()
    total_touches_by_match = touches_by_match_team.groupby(['Match_ID', 'MD'])['Touches_Att 3rd'].sum().reset_index()
    total_touches_by_match = total_touches_by_match.rename(columns={'Touches_Att 3rd': 'Total_Touches_Att 3rd'})
    field_tilt_df = pd.merge(touches_by_match_team, total_touches_by_match, on=['Match_ID', 'MD'])
    field_tilt_df['Field_Tilt'] = field_tilt_df['Touches_Att 3rd'] / field_tilt_df['Total_Touches_Att 3rd']

    # Calcula o Field Tilt acumulado até a rodada atual
    field_tilt_df['Cum_Touches_Att 3rd'] = field_tilt_df.groupby('Squad')['Touches_Att 3rd'].expanding().sum().reset_index(level=0, drop=True)
    field_tilt_df['Cum_Total_Touches_Att 3rd'] = field_tilt_df.groupby('Squad')['Total_Touches_Att 3rd'].expanding().sum().reset_index(level=0, drop=True)
    field_tilt_df['Field_Tilt_Over_Matches'] = field_tilt_df['Cum_Touches_Att 3rd'] / field_tilt_df['Cum_Total_Touches_Att 3rd']


    return field_tilt_df

Em termos de ordem de execução, o expanding é aplicado após os dados serem agrupados pelo groupby, mas antes da agregação final (como a soma, no nosso caso).

Veja como funciona:

1. groupby('Squad'): O DataFrame é dividido em grupos, um para cada equipe (Squad).

2. ['Touches_Att 3rd'].expanding().sum():
- Dentro de cada grupo (equipe), a função expanding cria uma janela expansiva que começa na primeira linha do grupo e se expande a cada linha subsequente.
- A função sum() calcula a soma dos valores dentro da janela expansiva para cada linha do grupo.
- Isso resulta em uma série de valores que representam a soma acumulada dos toques no terço de ataque para cada equipe até a rodada em questão, dentro de cada grupo.
3. O resultado do expanding().sum() é então atribuído a uma nova coluna no DataFrame original.


In [42]:
calculate_field_tilt(df).head()

,Match_ID,Squad,MD,Touches_Att 3rd,Total_Touches_Att 3rd,Field_Tilt,Cum_Touches_Att 3rd,Cum_Total_Touches_Att 3rd,Field_Tilt_Over_Matches
0,1,Aston Villa,1,162,258,0.627907,162.0,258.0,0.627907
1,1,Young Boys,1,96,258,0.372093,96.0,258.0,0.372093
2,2,Juventus,1,89,309,0.288026,89.0,309.0,0.288026
3,2,PSV Eindhoven,1,220,309,0.711974,220.0,309.0,0.711974
4,3,Lille,1,119,233,0.510730,119.0,233.0,0.510730


In [35]:
calculate_field_tilt(df)[['Match_ID', 'Squad', 'Field_Tilt']].sort_values(by=['Field_Tilt'], ascending=False).head()

,Match_ID,Squad,Field_Tilt
52,27,Manchester City,0.946640
96,49,Manchester City,0.918200
284,143,Bayern Munich,0.910180
242,122,Bayern Munich,0.902985
10,6,Bayern Munich,0.889213


In [36]:
ucl2425_fieldtilt = calculate_field_tilt(df)

In [37]:
# Multiplica as colunas por 100 para ter os valores em porcentagem
ucl2425_fieldtilt['Field_Tilt'] = ucl2425_fieldtilt['Field_Tilt'] * 100
ucl2425_fieldtilt['Field_Tilt_Over_Matches'] = ucl2425_fieldtilt['Field_Tilt_Over_Matches'] * 100

# Renomeia as colunas incluindo "%" no nome
ucl2425_fieldtilt = ucl2425_fieldtilt.rename(columns={
    'Field_Tilt': 'Field_Tilt%',
    'Field_Tilt_Over_Matches': 'Field_Tilt_Over_Matches%'
})

# Arredonda os valores para duas casas decimais, mantendo o tipo numérico
ucl2425_fieldtilt['Field_Tilt%'] = ucl2425_fieldtilt['Field_Tilt%'].round(2)
ucl2425_fieldtilt['Field_Tilt_Over_Matches%'] = ucl2425_fieldtilt['Field_Tilt_Over_Matches%'].round(2)

In [38]:
pd.set_option('display.max_rows', None)  # Mostrar todas as linhas
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas

In [39]:
ucl2425_fieldtilt.sort_values(by=['Field_Tilt%'], ascending=False).head(10)

,Match_ID,Squad,MD,Touches_Att 3rd,Total_Touches_Att 3rd,Field_Tilt%,Cum_Touches_Att 3rd,Cum_Total_Touches_Att 3rd,Field_Tilt_Over_Matches%
52,27,Manchester City,2,479,506,94.66,800.0,881.0,90.81
96,49,Manchester City,3,449,489,91.82,1249.0,1370.0,91.17
284,143,Bayern Munich,8,456,501,91.02,2381.0,2843.0,83.75
242,122,Bayern Munich,7,363,402,90.30,1925.0,2342.0,82.19
10,6,Bayern Munich,1,305,343,88.92,305.0,343.0,88.92
112,57,Manchester City,4,281,323,87.00,1530.0,1693.0,90.37
257,129,Manchester City,8,370,426,86.85,2541.0,3102.0,81.91
130,66,Bayern Munich,4,377,435,86.67,1111.0,1320.0,84.17
92,47,Atalanta,3,390,451,86.47,777.0,1040.0,74.71
48,25,Barcelona,2,269,312,86.22,327.0,556.0,58.81


In [40]:
raise SystemExit

SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [43]:
file_path = '/content/drive/MyDrive/Dados/Projetos/UCL_2425/fieldtiltucl2425.csv'

ucl2425_fieldtilt.to_csv(file_path, index=False)